# diffusion

> Fill in a module description here

In [ ]:
#| default_exp diffusion

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchvision

from diffusers import DDPMScheduler, UNet2DModel
from matplotlib import pyplot as plt
from fastcore.foundation import L

from foundation.utils import show_img

/Users/education/miniforge3/envs/sb-from-scratch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print(f'Using device: {device}')

In [ ]:
# mnist_dataset = torchvision.datasets.MNIST(root="mnist/", train=True, download=True, transform=torchvision.transforms.ToTensor())

In [ ]:
# train_dataloader = DataLoader(mnist_dataset, batch_size=8, shuffle=True)

In [ ]:
# x, y = next(iter(train_dataloader))
# print('Input shape:', x.shape)
# print('Labels:', y)
# plt.imshow(torchvision.utils.make_grid(x)[0], cmap='Greys');

### Why Diffusion Process

The idea of diffusion is we sequentially add more noise to an image and then lets the model to predict what's noise it need to remove to get back the original image

### Diffusion Process

In [ ]:
# img = next(iter(train_dataloader))[0][1][0]

In [ ]:
# show_img(img)

In [ ]:
# noise = torch.randn_like(img)

In [ ]:
# show_img(img + noise)

In [ ]:
# #| export
# def corrupt(img, amount):
#     noise = torch.randn_like(img)
#     return (1-amount)*img + noise * amount

In [ ]:
# amounts = torch.linspace(0, 1, steps=10)

In [ ]:
# amounts

In [ ]:
# corrupt_images = []

In [ ]:
# for amount in amounts:
#     # ipdb.set_trace()
#     corrupt_images.append(corrupt(img, amount))

In [ ]:
# for corrupt_imgage in corrupt_images:
#     show_img(corrupt_imgage)

In [ ]:
# torchvision.utils.make_grid(corrupt_images).shape

In [ ]:
# fig, axs = plt.subplots(1, 1, figsize=(12, 5))

# axs.set_title('Input data')
# axs.imshow(torchvision.utils.make_grid(corrupt_images)[0], cmap='Greys')
# axs.imshow(torchvision.utils.make_grid(corrupt_images)[1], cmap='Greys')

In [ ]:
# axs.imshow(torchvision.utils.make_grid(corrupt_images)[5], cmap='Greys')

In [ ]:
# #| export

# class BasicUNet(nn.Module):
#     """A minimal UNet implementation."""
#     def __init__(self, in_channels, out_channels):
#         super().__init__()
#         self.down_layers = torch.nn.ModuleList([ 
#             nn.Conv2d(in_channels, 32, kernel_size=5, padding=2),
#             nn.Conv2d(32, 64, kernel_size=5, padding=2),
#             nn.Conv2d(64, 64, kernel_size=5, padding=2),
#         ])
#         self.up_layers = torch.nn.ModuleList([
#             nn.Conv2d(64, 64, kernel_size=5, padding=2),
#             nn.Conv2d(64, 32, kernel_size=5, padding=2),
#             nn.Conv2d(32, out_channels, kernel_size=5, padding=2), 
#         ])
#         self.act = nn.SiLU()
#         self.downscale = nn.MaxPool2d(2)
#         self.upscale = nn.Upsample(scale_factor=2)

#     def forward(self, x):
#         h = []
#         for i, l in enumerate(self.down_layers):
#             x = self.act(l(x))
#             h.append(x)
#             if i < 2: x = self.downscale(x)
#         for i, l in enumerate(self.up_layers):
#             if i > 0: x = self.upscale(x)
#             x += h.pop()
#             x = self.act(l(x))
#         return x

In [ ]:
# unet = BasicUNet(1, 2)
# # unet(torch.rand(8, 1, 28, 28)).shape

In [ ]:
# net = BasicUNet(1, 1)

# # Feed some data through:
# x.shape, net(x).shape

In [ ]:
# loss_fn = nn.MSELoss()

# # The optimizer - explore different learning rates or try
# # a different optimizer instead
# opt = torch.optim.Adam(net.parameters(), lr=3e-4) 

# # Keeping a record of the losses for later viewing
# losses = []

# # And a record of smoothed loss values after each epoch
# smoothed_losses_basic = []